In [1]:
from BCEmbedding.tools.langchain import BCERerank

/slurm/home/admin/.conda/envs/dl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/slurm/home/admin/.conda/envs/dl/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
# FAISS是一个高效的相似性搜索库，常用于大规模向量搜索。
from langchain_community.vectorstores.faiss import FAISS

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.retrievers import ContextualCompressionRetriever

In [4]:
# 初始化embedding模型

embedding_model_name = "/slurm/resources/weights/huggingface/maidalun1020/bce-embedding-base_v1"
embedding_model_kwargs = {"device": "cuda:0"}
embedding_encode_kwargs = {"batch_size": 32,
                           "normalize_embeddings": True}

In [5]:
embed_model = HuggingFaceEmbeddings(
    model_name = embedding_model_name,
    model_kwargs = embedding_model_kwargs,
    encode_kwargs = embedding_encode_kwargs
)

/slurm/home/admin/.conda/envs/dl/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
07/25/2024 10:27:38 - [INFO] -datasets->>>    PyTorch version 2.2.0 available.
/slurm/home/admin/.conda/envs/dl/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
07/25/2024 10:27:38 - [INFO] -sentence_transformers.SentenceTransformer->>>    Load pretrained SentenceTransformer: /slurm/resources/weights/huggingface/maidalun1020/bce-embedd

In [6]:
# 初始化rereanker模型

reranker_args = {
    "model": "/slurm/resources/weights/huggingface/maidalun1020/bce-reranker-base_v1/",
    "top_n": 5,
    "device": "cuda:0"
}

reranker = BCERerank(**reranker_args)

07/25/2024 10:28:02 - [INFO] -BCEmbedding.models.RerankerModel->>>    Loading from `/slurm/resources/weights/huggingface/maidalun1020/bce-reranker-base_v1/`.
07/25/2024 10:28:02 - [INFO] -BCEmbedding.models.RerankerModel->>>    Execute device: cuda:0;	 gpu num: 1;	 use fp16: False


In [10]:
# 文档数据预处理

documents = UnstructuredPDFLoader("./RAG-编年史.pdf").load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                               chunk_overlap=200)

texts = text_splitter.split_documents(documents)

07/25/2024 10:33:23 - [INFO] -pikepdf._core->>>    pikepdf C++ to Python logger bridge initialized


In [11]:
texts[0]

Document(metadata={'source': './RAG-编年史.pdf'}, page_content='唐国梁Tommy\n\n唐国梁Tommy\n\n唐国梁Tommy\n\n1. 论⽂名称：the chronicles of rag: the retriever, the chunk and the generator\n\nRAG编年史：检索器，⽂本块和⽣成器\n\nAbstract\n\n背景：\n\n唐国梁Tommy\n\nRAG是⼀种使⼤型语⾔模型（LLMs）访问外部数据的流⾏范式。\n\n唐国梁Tommy\n\nRAG的实施⾯临诸多挑战，包括检索模型的有效整合、表示学习、数据多样性、计算效率优化等。\n\n唐国梁Tommy\n\n⽬的：\n\n提出对巴⻄葡萄⽛语实施、优化和评估RAG的良好实践。\n\n通过简单的流⽔线进⾏推理和实验，回答有关哈利·波特书籍的问题。\n\n⽅法：\n\n使⽤OpenAI的gpt-4、gpt-4-1106-preview、gpt-3.5-turbo-1106和Google的Gemini Pro⽣成答案。\n\n唐国梁Tommy\n\n重点提升检索器的质量。\n\n唐国梁Tommy\n\n结果：\n\n唐国梁Tommy\n\n⽅法使得MRR@10相⽐基线提⾼了35.4%。\n\n经过优化，性能进⼀步提升了2.4%。\n\n结论：\n\nRAG架构经过建议的增强后，相对得分从57.88%提⾼到了98.61%的最⼤值。\n\n唐国梁Tommy\n\n1 Introduction\n\n唐国梁Tommy\n\n唐国梁Tommy\n\n背景：\n\nLLMs在AI应⽤中取得显著成绩，尤其在翻译、总结等任务中。\n\n需要基于最新信息和外部数据提供答案的问题上存在挑战。\n\n挑战：\n\n实现RAG技术⾯临新挑战，包括发展可靠检索器和确保检索⽂本相关性。\n\n唐国梁Tommy\n\nRAG领域发展：\n\n唐国梁Tommy\n\nRAG研究快速扩张，新论⽂介绍了不同的实现⽅式和技术改进。\n\n这些发展给AI实践者评估性能和适⽤性带来挑战。\n\n研究内容：\n\n针对巴⻄葡萄⽛语的RAG实施的全⾯实验。\n\n评估不同检索技术，探讨分块策略优化检索信息的整合。\n\n唐国梁Tommy\n\n唐国梁Tommy\n\

In [12]:
print(len(texts))

8


In [13]:
print(texts[0])

page_content='唐国梁Tommy

唐国梁Tommy

唐国梁Tommy

1. 论⽂名称：the chronicles of rag: the retriever, the chunk and the generator

RAG编年史：检索器，⽂本块和⽣成器

Abstract

背景：

唐国梁Tommy

RAG是⼀种使⼤型语⾔模型（LLMs）访问外部数据的流⾏范式。

唐国梁Tommy

RAG的实施⾯临诸多挑战，包括检索模型的有效整合、表示学习、数据多样性、计算效率优化等。

唐国梁Tommy

⽬的：

提出对巴⻄葡萄⽛语实施、优化和评估RAG的良好实践。

通过简单的流⽔线进⾏推理和实验，回答有关哈利·波特书籍的问题。

⽅法：

使⽤OpenAI的gpt-4、gpt-4-1106-preview、gpt-3.5-turbo-1106和Google的Gemini Pro⽣成答案。

唐国梁Tommy

重点提升检索器的质量。

唐国梁Tommy

结果：

唐国梁Tommy

⽅法使得MRR@10相⽐基线提⾼了35.4%。

经过优化，性能进⼀步提升了2.4%。

结论：

RAG架构经过建议的增强后，相对得分从57.88%提⾼到了98.61%的最⼤值。

唐国梁Tommy

1 Introduction

唐国梁Tommy

唐国梁Tommy

背景：

LLMs在AI应⽤中取得显著成绩，尤其在翻译、总结等任务中。

需要基于最新信息和外部数据提供答案的问题上存在挑战。

挑战：

实现RAG技术⾯临新挑战，包括发展可靠检索器和确保检索⽂本相关性。

唐国梁Tommy

RAG领域发展：

唐国梁Tommy

RAG研究快速扩张，新论⽂介绍了不同的实现⽅式和技术改进。

这些发展给AI实践者评估性能和适⽤性带来挑战。

研究内容：

针对巴⻄葡萄⽛语的RAG实施的全⾯实验。

评估不同检索技术，探讨分块策略优化检索信息的整合。

唐国梁Tommy

唐国梁Tommy

唐国梁Tommy

唐国梁Tommy

分析⽂档位置对⽣成内容质量的影响。

⽐较不同LLMs在整合检索信息产⽣回应的能⼒。

主要贡献：

1. 数据集准备⽅法，量化RAG系统不同步骤的质量。

2. 最⼤相对分数度量，量化⽅法与理想RAG系统间的差距。

3. 开发R

In [14]:
# FAISS.from_documents(...) 从给定的文档集合texts中创建一个FAISS索引

# distrance_strategy : 指定了距离计算策略为最大内积（MAX_INNER_PRODUCT）。
# 在相似性搜索中，内积常用于衡量向量之间的相似度，最大内积策略意味着会优先考虑内积值最大的项，即最相似的文档。

# as_retriever() : 将之前创建的FAISS索引转化为一个检索器，用于执行相似性搜索。
# score_threshold=0.3表示只有当相似性得分超过0.3时，结果才会被认为是相关的，这是一个过滤结果的阈值。
# "k": 10指定了返回结果的数量，即对于每个查询，返回得分最高的10个文档。

retriever = FAISS.from_documents(texts, 
                                 embed_model, 
                                 distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT
                                ).as_retriever(search_type="similarity", 
                                               search_kwargs={"score_threshold": 0.3, "k": 10})

07/25/2024 10:36:13 - [INFO] -faiss.loader->>>    Loading faiss with AVX2 support.
07/25/2024 10:36:13 - [INFO] -faiss.loader->>>    Successfully loaded faiss with AVX2 support.


In [15]:
# 首先使用retriever来进行初始的文档检索，然后可能使用base_compressor（即reranker）来进一步处理这些检索结果，
# 比如通过重新排序来提高检索结果的相关性。

compression_retriever = ContextualCompressionRetriever(
    base_compressor = reranker,
    base_retriever = retriever,
)

In [18]:
response = compression_retriever.get_relevant_documents("RAG是什么?")

Token indices sequence length is longer than the specified maximum sequence length for this model (854 > 512). Running this sequence through the model will result in indexing errors
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [19]:
response

[Document(metadata={'source': './RAG-编年史.pdf', 'relevance_score': 0.5261356234550476}, page_content='唐国梁Tommy\n\n唐国梁Tommy\n\n唐国梁Tommy\n\n1. 论⽂名称：the chronicles of rag: the retriever, the chunk and the generator\n\nRAG编年史：检索器，⽂本块和⽣成器\n\nAbstract\n\n背景：\n\n唐国梁Tommy\n\nRAG是⼀种使⼤型语⾔模型（LLMs）访问外部数据的流⾏范式。\n\n唐国梁Tommy\n\nRAG的实施⾯临诸多挑战，包括检索模型的有效整合、表示学习、数据多样性、计算效率优化等。\n\n唐国梁Tommy\n\n⽬的：\n\n提出对巴⻄葡萄⽛语实施、优化和评估RAG的良好实践。\n\n通过简单的流⽔线进⾏推理和实验，回答有关哈利·波特书籍的问题。\n\n⽅法：\n\n使⽤OpenAI的gpt-4、gpt-4-1106-preview、gpt-3.5-turbo-1106和Google的Gemini Pro⽣成答案。\n\n唐国梁Tommy\n\n重点提升检索器的质量。\n\n唐国梁Tommy\n\n结果：\n\n唐国梁Tommy\n\n⽅法使得MRR@10相⽐基线提⾼了35.4%。\n\n经过优化，性能进⼀步提升了2.4%。\n\n结论：\n\nRAG架构经过建议的增强后，相对得分从57.88%提⾼到了98.61%的最⼤值。\n\n唐国梁Tommy\n\n1 Introduction\n\n唐国梁Tommy\n\n唐国梁Tommy\n\n背景：\n\nLLMs在AI应⽤中取得显著成绩，尤其在翻译、总结等任务中。\n\n需要基于最新信息和外部数据提供答案的问题上存在挑战。\n\n挑战：\n\n实现RAG技术⾯临新挑战，包括发展可靠检索器和确保检索⽂本相关性。\n\n唐国梁Tommy\n\nRAG领域发展：\n\n唐国梁Tommy\n\nRAG研究快速扩张，新论⽂介绍了不同的实现⽅式和技术改进。\n\n这些发展给AI实践者评估性能和适⽤性带来挑战。\n\n研究内容：\n\n针对巴⻄葡萄⽛语的RAG实施的全⾯实验。\n\n评估不同检索技术，探讨分

In [20]:
print(response[0].page_content)

唐国梁Tommy

唐国梁Tommy

唐国梁Tommy

1. 论⽂名称：the chronicles of rag: the retriever, the chunk and the generator

RAG编年史：检索器，⽂本块和⽣成器

Abstract

背景：

唐国梁Tommy

RAG是⼀种使⼤型语⾔模型（LLMs）访问外部数据的流⾏范式。

唐国梁Tommy

RAG的实施⾯临诸多挑战，包括检索模型的有效整合、表示学习、数据多样性、计算效率优化等。

唐国梁Tommy

⽬的：

提出对巴⻄葡萄⽛语实施、优化和评估RAG的良好实践。

通过简单的流⽔线进⾏推理和实验，回答有关哈利·波特书籍的问题。

⽅法：

使⽤OpenAI的gpt-4、gpt-4-1106-preview、gpt-3.5-turbo-1106和Google的Gemini Pro⽣成答案。

唐国梁Tommy

重点提升检索器的质量。

唐国梁Tommy

结果：

唐国梁Tommy

⽅法使得MRR@10相⽐基线提⾼了35.4%。

经过优化，性能进⼀步提升了2.4%。

结论：

RAG架构经过建议的增强后，相对得分从57.88%提⾼到了98.61%的最⼤值。

唐国梁Tommy

1 Introduction

唐国梁Tommy

唐国梁Tommy

背景：

LLMs在AI应⽤中取得显著成绩，尤其在翻译、总结等任务中。

需要基于最新信息和外部数据提供答案的问题上存在挑战。

挑战：

实现RAG技术⾯临新挑战，包括发展可靠检索器和确保检索⽂本相关性。

唐国梁Tommy

RAG领域发展：

唐国梁Tommy

RAG研究快速扩张，新论⽂介绍了不同的实现⽅式和技术改进。

这些发展给AI实践者评估性能和适⽤性带来挑战。

研究内容：

针对巴⻄葡萄⽛语的RAG实施的全⾯实验。

评估不同检索技术，探讨分块策略优化检索信息的整合。

唐国梁Tommy

唐国梁Tommy

唐国梁Tommy

唐国梁Tommy

分析⽂档位置对⽣成内容质量的影响。

⽐较不同LLMs在整合检索信息产⽣回应的能⼒。

主要贡献：

1. 数据集准备⽅法，量化RAG系统不同步骤的质量。

2. 最⼤相对分数度量，量化⽅法与理想RAG系统间的差距。

3. 开发RAG系统时的最佳实践和优化讨